# Clean Data

* Not all the comments in the thread are job postings. Some are questions and comments too. I could see the conversations that happen around the posting containing useful information, but 
* mark posts that aren't job postings

In [22]:
#| default_exp clean
from hn_jobs_chat.init_scrape import postsTableName


We need to determine:
* company name
* company location
* contact info

If any information isn't present, we need to make sure that we don't hallucinate.

## Is the posting a job posting?

Add the `is_job` flag to the table

In [4]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

query = """ALTER TABLE IF EXISTS """ + postsTableName + """ ADD COLUMN IF NOT EXISTS is_job BOOL;"""

cursor.execute(query)

conn.commit()   
conn.close() 

We call an LLM, and have it evaluate the job descriptions, marking the ones that are not jobs for us. 

In [5]:
# |export
import json
import os
import openai

openai.api_key = os.environ.get('OPENAI_API_KEY')

def getResponse(text: str):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": text}],
    )

    return response.choices[0].message.content

def toJsonOrReprompt(item, response, repromptNum=None):

    print('response', response)
    # try catch here. If item fails, rerun it. If it fails more than 3 times, return None
    try:
        return json.loads(response.replace('\n',''))
    except:
        if repromptNum == None:
            repromptNum = 0
        
        if repromptNum > 3:
            return { "error": "Could not receive valid json: Too many reprompts","item": '+item+'}
        
        repromptNum += 1
        return reprompt(response, repromptNum)
    
def reprompt(item, prevResponse, repromptNum=None):
    newPrompt = 'Your response: \n\n'+prevResponse.text+'\n\nwas not valid json. Please try again. Here is the original request:\n' + item

    response = getResponse(newPrompt)

    return toJsonOrReprompt(item, response, repromptNum=repromptNum)


def isJob(item):

    # If the comment is empty, it is not a job posting. We do not need LLM to determine this.
    if item['comment'] == "":
        return False

    text="""
        Is the following message a job posting, or something else? If it is a job posting, reply { "isPosting": true }. If it is not a job posting, reply with { "isPosting": false }.\n-----------------\n
    """+ item['comment']

    result = getResponse(text)
    resp = toJsonOrReprompt(item, result)
 
    if 'error' in resp.keys():
        print (resp['error'])
    else:
        return resp['isPosting']



In [6]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

query = """SELECT * FROM """ + postsTableName + """ WHERE is_job IS NULL;"""

cursor.execute(query)

rows = cursor.fetchall()

for row in rows:

    id, name, date, comment, fileName,  html, is_job = row

    item = {
        "id": id,
        "comment": comment
    }

    resultItem = isJob(item)

    print(resultItem)

    if resultItem is not None:
        query = """UPDATE """ + postsTableName + """ SET is_job = """ + str(resultItem) + """ WHERE id = """ + str(item['id']) + """;"""
        cursor.execute(query)


conn.commit()
conn.close()


response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true }
True
response { "isPosting": true

Ok, let's look at the disqualified posts. 

In July's set, they were all deleted posts except one: it also picked out a position that was closed. Which, is technically, not a job posting per se, even though it was at one point.

At first I though this was a little suspicious, given that there are always comments in the threads. But, then I remembered that we dropped them initially: since replies are all indented, by dropping indented posts during scaping, we dropped all the replies. So, it makes sense that all would be left top-level comments - which I'd expect to be deleted, or commented as closed. 

However, it looks like every top level comment this month was a job posting - at least when I scraped the data. While this is entirely plausible, it's worth investigating. 


In [25]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

query = """SELECT * FROM """ + postsTableName + """ WHERE is_job IS FALSE;"""
cursor.execute(query)

rows = cursor.fetchall()

for row in rows:

        id, name, date, comment, fileName,  html, is_job = row

        print(id, name, comment)


In [ ]:
# | hide
from nbdev import nbdev_export

nbdev_export()